In [34]:
import pandas as pd
import numpy as np
import os
import re

In [35]:
files = [f"1_CG_GA/reports/{f}" for f in os.listdir("1_CG_GA/reports") if (("processed" not in f) and ("extra" not in f) and ("tmax10" not in f))]
files

['1_CG_GA/reports/1_CG_GA_10_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_11_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_12_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_13_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_14_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_15_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_16_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_17_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_18_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_19_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_1_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_20_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_21_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_22_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_23_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_24_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_25_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_26_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_27_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_28_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_29_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_2_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_30_tmax15.csv',
 '1_CG_GA/reports/1_CG_GA_3_tmax15.c

In [36]:
import pandas as pd
import re
import glob # Assuming you use glob to get the file list

# Assuming 'files' is a list of your csv file paths, e.g.,
# files = glob.glob('path/to/your/files/*.csv')

final_df = pd.DataFrame()

for f in files:
    df = pd.read_csv(f)
    pattern = r'_(\d+)'
    match = re.search(pattern, f)

    # 3. Atribuir o nome do experimento
    exp_name = df.experiment_name.values[0] #Default value
    if match:
        # If the pattern is found, the result is the captured group
        exp_name = match.group(1) # Using group(1) is better to get just the number

    df["experiment_name"] = exp_name # No need to create a list here

    # --- START OF FIX ---

    # # 1. Filter the DataFrame first
    # time_df = df[df.metric.str.contains("time")].copy()

    # # 2. KEY FIX: Convert the 'value' column to a numeric type.
    # #    errors='coerce' will turn any values that can't be converted into NaN
    # time_df['value'] = pd.to_numeric(time_df['value'], errors='coerce')

    # 3. Now, perform the pivot operation on the cleaned DataFrame
    pivoted_df = df.pivot_table(
        index=["experiment_name"],
        columns=["metric"],
        values="value"
    ).reset_index()

    # --- END OF FIX ---

    final_df = pd.concat([final_df, pivoted_df])

print(final_df)
final_df.to_csv("processing_times_report_GC_GA.csv", index=False)

metric experiment_name      best_cost  best_fitness  best_r  best_s  best_u  \
0                   10  119917.273803      8.960573    20.0    42.0    54.0   
0                   11  112827.112989     12.254902     3.0    33.0    35.0   
0                   12  111378.477106     11.682243     1.0    37.0    41.0   
0                   13  111451.640695     12.106538     0.0    37.0    37.0   
0                   14  115799.485060     11.961722     0.0    41.0    32.0   
0                   15  115375.834865      7.874016    31.0    34.0    79.0   
0                   16  111181.217486     12.500000     0.0    35.0    35.0   
0                   17  112183.450953      9.487666    12.0    37.0    63.0   
0                   18  114285.133677      6.402049    46.0    32.0   114.0   
0                   19  114566.458532     11.600928     0.0    40.0    37.0   
0                    1  110082.841750      9.980040    15.0    33.0    57.0   
0                   20  113385.590143      7.462687 

COLUMN GENERATION METRICS

In [74]:
import re

# Supondo que 'files' seja uma lista de seus arquivos CSV
# files = ["exp_1.csv", "exp_2.csv", ...] 

final_df = pd.DataFrame()

for f in files:
    df = pd.read_csv(f)
    pattern = r'_(\d+)\.csv$'
    match = re.search(pattern, f)
    
    # Atribuir o nome do experimento
    exp_name = "unknown" # Valor padrão caso o padrão não seja encontrado
    if match:
        # Usar o nome completo do arquivo ou apenas o número, dependendo da sua necessidade
        # group(0) -> _123.csv
        # group(1) -> 123
        exp_name = match.group(1) # Extraindo apenas o número do experimento

    df["experiment_name"] = [exp_name] * len(df) # Corrigido para funcionar com qualquer tamanho de df
    
    # Filtrando os dados
    df = df[df.step.str.contains("column") & ~df.metric.str.contains("time")]

    # --- SOLUÇÃO AQUI ---
    # Converta a coluna 'value' para um tipo numérico.
    # Se algum valor não puder ser convertido (ex: texto), ele se tornará NaN.
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    # Agora o pivot_table funcionará
    df_pivot = df.pivot_table(index=["experiment_name"], columns=["metric"], values="value").reset_index()
    
    final_df = pd.concat([final_df, df_pivot])

print(final_df)
final_df.to_csv("column_generation_report_pool_GA_CG_fixing.csv", index=False)

metric experiment_name  final_objective  number_of_iterations  \
0                    1    106977.465562                 331.0   
0                   10    108381.655903                 317.0   
0                   11    111048.289285                1477.0   
0                   12    107622.657160                 327.0   
0                   13    109891.519704                 355.0   
0                   14    111175.558545                1201.0   
0                   15    111705.295327                 388.0   
0                   16    143991.174086                   3.0   
0                   17    106981.876519                1470.0   
0                   18    112108.223744                1205.0   
0                    2    147758.829172                   3.0   
0                    3    110996.318289                 369.0   
0                    4    105803.637124                 317.0   
0                    5    112090.669598                 441.0   
0                    6   

GENETIC ALGORITHM

In [75]:
files = [f"5_pool_GA_CG/reports/{f}" for f in os.listdir("5_pool_GA_CG/reports") if "mipFalse" in f]
files

['5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_1.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_10.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_11.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_12.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_13.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_14.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_15.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_16.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_17.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_18.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_2.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_3.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_4.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_5.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_6.csv',
 '5_pool_GA_CG/r

In [76]:
import re

# Supondo que 'files' seja uma lista de nomes de arquivos
# files = ["exp_run_123.csv", "exp_run_456.csv"] 

final_df = pd.DataFrame()

for f in files:
    df = pd.read_csv(f)
    pattern = r'_(\d+)\.csv$'
    match = re.search(pattern, f)

    exp_name = "unknown_experiment" # Valor padrão caso o padrão não seja encontrado
    if match:
        # Usar group(1) para pegar apenas os dígitos, que parece ser a intenção.
        # Se quiser '_123.csv', use match.group(0) como no original.
        exp_name = match.group(1) 

    df["experiment_name"] = exp_name # Atribuição mais eficiente

    # Filtrando o DataFrame
    df_filtered = df[df.step.str.contains("genetic_algorithm") & ~df.metric.str.contains("time")].copy()
    
    # --- SOLUÇÃO PRINCIPAL AQUI ---
    # Converta a coluna 'value' para um formato numérico.
    # 'errors='coerce'' irá transformar entradas inválidas (ex: texto) em NaN
    df_filtered['value'] = pd.to_numeric(df_filtered['value'], errors='coerce')

    # Opcional: remova linhas onde a conversão falhou (resultou em NaN)
    df_filtered.dropna(subset=['value'], inplace=True)
    
    # Verificação (bom para depuração, pode ser removido depois)
    # print(f"Processando arquivo: {f}")
    # print(df_filtered.info())
    # print(df_filtered.head())

    if not df_filtered.empty:
        df_pivoted = df_filtered.pivot_table(
            index=["experiment_name"],
            columns=["metric"], 
            values="value"
        ).reset_index()
        final_df = pd.concat([final_df, df_pivoted])

print(final_df)
final_df.to_csv("ga_report_pool_GA_CG_fixing.csv", index=False)

metric experiment_name     best_cost  best_fitness  best_r  best_s  best_u  \
0                    1  1.558287e+06    637.784212  2978.0   763.0     0.0   
0                   10  1.496552e+06    699.244174  2780.0   769.0     0.0   
0                   11  1.536685e+06    677.222585  2873.0   787.0     0.0   
0                   12  1.515405e+06    763.566332  2783.0   793.0     0.0   
0                   13  1.468988e+06    588.749031  2698.0   773.0     0.0   
0                   14  1.486741e+06    627.660867  2767.0   753.0     0.0   
0                   15  1.518864e+06    788.395604  2800.0   759.0     0.0   
0                   16  1.480147e+06    661.162732  2719.0   775.0     0.0   
0                   17  1.534199e+06    569.068037  2884.0   806.0     0.0   
0                   18  1.490137e+06    786.820389  2713.0   788.0     0.0   
0                    2  1.455447e+06    625.155238  2703.0   711.0     0.0   
0                    3  1.507974e+06    681.298135  2784.0   782

In [77]:
files = [f"5_pool_GA_CG/reports/{f}" for f in os.listdir("5_pool_GA_CG/reports") if "mipFalse" in f]
files

['5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_1.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_10.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_11.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_12.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_13.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_14.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_15.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_16.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_17.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_18.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_2.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_3.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_4.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_5.csv',
 '5_pool_GA_CG/reports/5_pool_GA_CG_20Q_09theta_mipFalse_6.csv',
 '5_pool_GA_CG/r

In [78]:
import pandas as pd
import os
import glob

def rename_ga_blocks(df):
    """
    Identifica blocos de genetic_algorithm e renomeia para first_ga e second_ga
    """
    df = df.copy()
    
    # Encontrar todas as linhas de genetic_algorithm
    ga_mask = df['step'] == 'genetic_algorithm'
    
    if not ga_mask.any():
        return df
    
    # Extrair parte numérica dos log_ids para identificar grupos
    ga_rows = df[ga_mask].copy()

    # --- LINHA CORRIGIDA ---
    # Extrai a parte numérica da string e converte para float para evitar OverflowError
    ga_rows['log_id_base'] = ga_rows['log_ids'].astype(str).str.extract(r'(\d+)')[0].astype(float)
    
    # Identificar quebras significativas nos log_ids (indicando nova execução do GA)
    ga_rows['id_diff'] = ga_rows['log_id_base'].diff()
    
    # Uma diferença grande nos IDs indica uma nova execução
    # Vamos usar um threshold baseado na observação dos dados
    threshold = 10000000  # 10 milhões - ajustar se necessário
    
    # Marcar onde começam novos blocos de GA
    new_block_mask = (ga_rows['id_diff'] > threshold) | (ga_rows['id_diff'].isna())
    ga_rows['block_number'] = new_block_mask.cumsum() - 1
    
    # Renomear as steps baseado no número do bloco
    for idx, row in ga_rows.iterrows():
        if row['block_number'] == 0:
            df.loc[idx, 'step'] = 'first_ga'
        elif row['block_number'] == 1:
            df.loc[idx, 'step'] = 'second_ga'
        else:
            df.loc[idx, 'step'] = f'ga_{int(row["block_number"]) + 1}'
    
    return df

def add_step_numbering(df):
    """
    Adiciona numeração sequencial às steps mantendo o mesmo número para steps do mesmo tipo
    """
    df = df.copy()
    
    # Identificar a ordem das steps únicas mantendo a sequência original
    step_order = []
    seen_steps = set()
    
    for step in df['step']:
        if step not in seen_steps:
            step_order.append(step)
            seen_steps.add(step)
    
    # Criar mapeamento de step para número
    step_to_number = {step: i + 1 for i, step in enumerate(step_order)}
    
    # Aplicar numeração
    df['numbered_step'] = df['step'].map(step_to_number).astype(str) + '_' + df['step']
    df['step'] = df['numbered_step']
    df.drop('numbered_step', axis=1, inplace=True)
    
    return df

def process_csv_file(file_path):
    """
    Processa um arquivo CSV aplicando as transformações solicitadas:
    1. Substitui experiment_name baseado no nome do arquivo
    2. Renomeia blocos de GA quando há múltiplas execuções
    3. Adiciona numeração sequencial às steps
    """
    
    # Ler o CSV
    df = pd.read_csv(file_path)
    
    # 1. Transformar o nome do experimento baseado no arquivo
    filename = file_path.split("/")[-1]
    filename = filename.split(".")[-2]  # Remove extensão
    exp_name = "_".join(filename.split("_")[0:-1])
    
    df["experiment_name"] = exp_name
    
    # 2. Identificar e renomear blocos de GA
    df = rename_ga_blocks(df)
    
    # 3. Adicionar numeração sequencial às steps
    df = add_step_numbering(df)
    
    return df

def process_single_csv(file_path, save_output=True):
    """
    Função principal simplificada para processar um único arquivo CSV
    
    Args:
        file_path (str): Caminho para o arquivo CSV
        save_output (bool): Se deve salvar o arquivo processado
    
    Returns:
        DataFrame: DataFrame processado
    """
    print(f"Processando arquivo: {file_path}")
    
    # Processar arquivo
    df = process_csv_file(file_path)
    
    print(f"Dados processados: {len(df)} linhas")
    
    # Mostrar informações do processamento
    filename = file_path.split("/")[-1].split(".")[-2]
    exp_name = "_".join(filename.split("_")[0:-1])
    print(f"Novo experiment_name: {exp_name}")
    
    # Contar blocos de GA
    first_ga_count = len(df[df['step'].str.contains('first_ga', na=False)])
    second_ga_count = len(df[df['step'].str.contains('second_ga', na=False)])
    
    if first_ga_count > 0:
        print(f"Blocos GA identificados - first_ga: {first_ga_count}, second_ga: {second_ga_count}")
    
    # Mostrar steps finais
    unique_steps = df['step'].unique()
    print(f"Steps finais: {list(unique_steps)}")
    
    # Salvar se solicitado
    if save_output:
        output_path = file_path.replace('.csv', '_processed.csv')
        df.to_csv(output_path, index=False)
        print(f"Arquivo salvo: {output_path}")
    
    return df

def process_multiple_files(file_paths):
    """
    Processa múltiplos arquivos CSV
    """
    processed_dfs = []
    successful = 0
    failed = 0
    
    for file_path in file_paths:
        df_processed = process_single_csv(file_path)
        if df_processed is not None:
            processed_dfs.append(df_processed)
            successful += 1
            print("✓ Processado com sucesso\n")
        else:
            failed += 1
            print("✗ Falhou no processamento\n")
    
    print(f"RESUMO: {successful} arquivos processados com sucesso, {failed} falharam")
    return processed_dfs

def process_multiple_files(file_paths):
    """
    Processa múltiplos arquivos CSV
    """
    processed_dfs = []
    successful = 0
    failed = 0
    
    for file_path in file_paths:
        df_processed = process_single_csv(file_path)
        if df_processed is not None:
            processed_dfs.append(df_processed)
            successful += 1
            print("✓ Processado com sucesso\n")
        else:
            failed += 1
            print("✗ Falhou no processamento\n")
    
    print(f"RESUMO: {successful} arquivos processados com sucesso, {failed} falharam")
    return processed_dfs

# ===================== EXEMPLOS DE USO =====================

if __name__ == "__main__":
    
    # EXEMPLO 1: Processar um arquivo único
    # df = process_single_csv("caminho/para/seu_arquivo.csv")
    
    # EXEMPLO 2: Processar múltiplos arquivos de uma pasta
    arquivos = glob.glob("1_CG_GA/reports/*.csv")
    dfs_processados = process_multiple_files(arquivos)
    
    # EXEMPLO 3: Processar sem salvar automaticamente
    # df = process_single_csv("arquivo.csv", save_output=False)
    
    # EXEMPLO 4: Processar todos os CSV do diretório atual
    # arquivos_locais = glob.glob("*.csv")
    # for arquivo in arquivos_locais:
    #     process_single_csv(arquivo)
    
    print("Para usar este código:")
    print("1. Arquivo único: process_single_csv('caminho/arquivo.csv')")
    print("2. Múltiplos arquivos: process_multiple_files(['arquivo1.csv', 'arquivo2.csv'])")
    print("3. Todos CSV da pasta: process_multiple_files(glob.glob('pasta/*.csv'))")

Processando arquivo: 1_CG_GA/reports\1_CG_GA_10_tmax10.csv
Dados processados: 18 linhas
Novo experiment_name: reports\1_CG_GA_10
Blocos GA identificados - first_ga: 7, second_ga: 0
Steps finais: ['1_initial_solution', '2_graph_build', '3_column_generation', '4_first_ga']
Arquivo salvo: 1_CG_GA/reports\1_CG_GA_10_tmax10_processed.csv
✓ Processado com sucesso

Processando arquivo: 1_CG_GA/reports\1_CG_GA_10_tmax10_processed.csv
Dados processados: 18 linhas
Novo experiment_name: reports\1_CG_GA_10_tmax10
Blocos GA identificados - first_ga: 7, second_ga: 0
Steps finais: ['1_1_initial_solution', '2_2_graph_build', '3_3_column_generation', '4_4_first_ga']
Arquivo salvo: 1_CG_GA/reports\1_CG_GA_10_tmax10_processed_processed.csv
✓ Processado com sucesso

Processando arquivo: 1_CG_GA/reports\1_CG_GA_10_tmax10_processed_processed.csv
Dados processados: 18 linhas
Novo experiment_name: reports\1_CG_GA_10_tmax10_processed
Blocos GA identificados - first_ga: 7, second_ga: 0
Steps finais: ['1_1_1_ini

In [39]:
import numpy as np

# Create a random number generator instance
rng = np.random.default_rng()

# Define the inclusive range
L_min = 10
L_max = 20

# Generate a random integer where high=L_max is INCLUSIVE
# The endpoint=True is the key!
random_number = rng.integers(low=L_min, high=L_max, endpoint=True)

print(f"A random integer between {L_min} and {L_max} (inclusive) is: {random_number}")

A random integer between 10 and 20 (inclusive) is: 11


In [40]:
weights = np.array([1/30] * 30)

In [41]:
rng = np.random.default_rng()
vehicles = rng.integers(2, 10, endpoint=True)                

random_indices = np.random.choice(30, size=vehicles, replace=False, p=weights)
random_indices

array([13, 22, 10, 17, 15, 26,  8,  4], dtype=int32)

In [42]:
random_indices = np.sort(random_indices)
random_indices

array([ 4,  8, 10, 13, 15, 17, 22, 26], dtype=int32)

In [43]:
a = ((random_indices > 7) & (random_indices < 27))
random_indices[a]

array([ 8, 10, 13, 15, 17, 22, 26], dtype=int32)

In [44]:
def array_to_gene_batches(array, gene_segments):
    sorted_array = np.array(array)
    num_elements = sorted_array.size
    
    # Calculate the number of full batches
    num_full_batches = num_elements // gene_segments
    
    # Reshape the array into full batches
    batches = sorted_array[:num_full_batches * gene_segments].reshape(num_full_batches, gene_segments)
    
    # Handle the remaining elements as the last batch
    remaining_elements = sorted_array[num_full_batches * gene_segments:]
    if remaining_elements.size > 0:
        batches = np.vstack((batches, remaining_elements))
        
    return batches

In [45]:
batches = array_to_gene_batches(random_indices, 20)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 20 and the array at index 1 has size 8

In [46]:
array_0 = np.array([])
array_1 = np.array([1,2])
array_2 = np.array([8,7,6])

In [47]:
np.empty(30)

array([0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.03333333, 0.03333333, 0.03333333,
       0.        , 0.03333333, 0.03333333, 0.        , 0.03333333,
       0.        , 0.03333333, 0.        , 0.03333333, 0.03333333,
       0.03333333, 0.03333333, 0.        , 0.03333333, 0.03333333,
       0.03333333, 0.        , 0.03333333, 0.03333333, 0.03333333])

In [48]:
conc = np.concatenate((array_0, array_1, array_2))
conc

array([1., 2., 8., 7., 6.])

In [49]:
isa = np.array([0,4])

In [50]:
conc[isa]

array([1., 6.])

In [51]:
conc[~isa]

array([6., 1.])

In [52]:
a = [0,2,3,12,3]
a[0:2]

[0, 2]

In [53]:
a = np.array([True, False, True, True])

In [54]:
import numpy as np

array1 = np.array([1, 5])
array2 = np.array([3, 5, 6, 7])
np.isin(array1, array2)

array([False,  True])

In [75]:
np.where(a)[0]

array([0, 2, 3])

In [4]:
import pandas as pd
import json

In [5]:
file = 'C:/Users/soare/Documents/MD-EVSP/experiments/5_pool_GA_CG/solutions/5_pool_GA_CG_20Q_09theta_mipFalse_26.json'

In [11]:
with open(file) as f:
    data = json.loads(f.read())
print(data)

{'Route_4895': {'Path': ['d1_4', 'l4_cp1_9', 'l4_cp2_44', 'l4_cp1_69', 'c2_4', 'l4_cp2_130', 'l4_cp1_161', 'l4_cp2_186', 'l4_cp1_209', 'd1_4'], 'Cost': 1663.676, 'Data': {'total_dh_dist': 0.0, 'total_dh_time': 0.0, 'total_travel_dist': 168.0, 'total_travel_time': 532.0, 'total_wait_time': 110.68000000000006, 'final_time': '2025-09-07 18:50:00', 'total_charge_time': 112.32}, 'var_value': 1.0}, 'Route_5982': {'Path': ['d1_4', 'l4_cp1_17', 'l4_cp2_54', 'l59_cp2_38', 'l60_cp1_53', 'l60_cp2_62', 'c1_4', 'l4_cp1_229', 'l4_cp2_250', 'l4_cp1_273', 'd1_4'], 'Cost': 1758.4072460371667, 'Data': {'total_dh_dist': 7.629381863909613, 'total_dh_time': 22.888145591728836, 'total_travel_dist': 169.4, 'total_travel_time': 509.0, 'total_wait_time': 246.2660187005721, 'final_time': '2025-09-07 22:19:00', 'total_charge_time': 163.845835707699}, 'var_value': 1.0}, 'Route_6607': {'Path': ['d1_4', 'l4_cp1_35', 'l60_cp2_28', 'l60_cp1_37', 'l4_cp2_116', 'l4_cp1_137', 'c2_4', 'l4_cp2_252', 'l4_cp1_277', 'd1_4'],

In [13]:
pd.DataFrame(data)

,Route_4895,Route_5982,Route_6607,Route_7088,Route_89,Route_1266,Route_1272,Route_1275,Route_1287,Route_1290,...,Route_1271,Route_1274,Route_1279,Route_1289,Route_5888,Route_6116,Route_6155,Route_7015,Route_3586,Route_4988
Path,"[d1_4, l4_cp1_9, l4_cp2_44, l4_cp1_69, c2_4, l...","[d1_4, l4_cp1_17, l4_cp2_54, l59_cp2_38, l60_c...","[d1_4, l4_cp1_35, l60_cp2_28, l60_cp1_37, l4_c...","[d1_4, l4_cp1_31, l60_cp2_30, l59_cp1_39, l59_...","[d1_4, l4_cp1_135, d1_4]","[d1_4, l4_cp1_1, l4_cp2_26, l4_cp1_53, l4_cp2_...","[d1_4, l4_cp1_5, l4_cp2_34, l4_cp1_61, l4_cp2_...","[d1_4, l4_cp1_7, l4_cp2_36, l4_cp1_63, l4_cp2_...","[d1_4, l4_cp1_13, l60_cp1_19, l60_cp2_26, l59_...","[d1_4, l4_cp1_15, l59_cp2_20, l59_cp1_27, l4_c...",...,"[d2_59, l60_cp1_1, l60_cp2_8, l60_cp1_17, l4_c...","[d2_59, l59_cp2_2, l60_cp1_5, l4_cp2_42, l4_cp...","[d2_59, l59_cp2_4, l59_cp1_9, l59_cp2_14, l4_c...","[d2_59, l59_cp2_6, l59_cp1_11, l4_cp1_45, l4_c...","[d2_60, l60_cp2_12, l59_cp1_25, l4_cp1_83, l4_...","[d2_60, l60_cp2_14, l4_cp1_91, l4_cp2_134, l4_...","[d2_60, l60_cp2_6, l59_cp1_19, l60_cp2_24, l60...","[d2_60, l60_cp2_4, l59_cp1_17, l59_cp2_24, l60...","[d2_60, l60_cp2_16, l60_cp1_23, l4_cp2_96, l59...","[d2_60, l60_cp2_10, l60_cp1_29, l4_cp2_106, l5..."
Cost,1663.676,1758.407246,1674.091117,1690.602565,947.04,1985.836,2006.516083,1825.752677,1915.295997,1806.132674,...,1970.25433,1922.601559,1828.858002,1778.248211,1730.227283,1723.387766,1219.523846,1785.510847,1752.812165,1473.059339
Data,"{'total_dh_dist': 0.0, 'total_dh_time': 0.0, '...","{'total_dh_dist': 7.629381863909613, 'total_dh...","{'total_dh_dist': 6.474575443821309, 'total_dh...","{'total_dh_dist': 6.786669585820267, 'total_dh...","{'total_dh_dist': 28.8, 'total_dh_time': 86.39...","{'total_dh_dist': 28.8, 'total_dh_time': 86.39...","{'total_dh_dist': 35.58666958582027, 'total_dh...","{'total_dh_dist': 6.477287721910654, 'total_dh...","{'total_dh_dist': 18.32835656360627, 'total_dh...","{'total_dh_dist': 20.77974593733564, 'total_dh...",...,"{'total_dh_dist': 43.26666958582027, 'total_dh...","{'total_dh_dist': 8.877287721910655, 'total_dh...","{'total_dh_dist': 21.537315730690388, 'total_d...","{'total_dh_dist': 22.161504014688305, 'total_d...","{'total_dh_dist': 6.786669585820267, 'total_dh...","{'total_dh_dist': 15.06274028686908, 'total_dh...","{'total_dh_dist': 6.675498381658087, 'total_dh...","{'total_dh_dist': 19.950996763316173, 'total_d...","{'total_dh_dist': 6.786669585820267, 'total_dh...","{'total_dh_dist': 10.33605144972988, 'total_dh..."
var_value,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
